In [ ]:
# Business Impact Analysis
def calculate_business_impact(y_true, y_pred):
    service_rank = {'Normal': 1, 'Urgent': 2, 'Critical': 3}
    tier_cost = 100  # $ per tier
    late_risk = 500  # $ late delivery cost
    
    overspend = sum((service_rank[pred] - service_rank[true]) * tier_cost 
                   for true, pred in zip(y_true, y_pred) if service_rank[pred] > service_rank[true])
    
    late_cost = sum(late_risk for true, pred in zip(y_true, y_pred) 
                   if service_rank[pred] < service_rank[true])
    
    correct = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    
    return {'overspend': overspend, 'late_cost': late_cost, 'correct': correct,
            'total_cost': overspend + late_cost}

baseline_impact = calculate_business_impact(y_test, y_pred_baseline)
best_impact = calculate_business_impact(y_test, best_predictions)

print("="*60)
print("BUSINESS IMPACT (Test Set)")
print("="*60)
print(f"\nBaseline:")
print(f"  Correct: {baseline_impact['correct']}/{len(y_test)} ({baseline_impact['correct']/len(y_test)*100:.1f}%)")
print(f"  Total cost: ${baseline_impact['total_cost']:,}")

print(f"\nBest Model:")
print(f"  Correct: {best_impact['correct']}/{len(y_test)} ({best_impact['correct']/len(y_test)*100:.1f}%)")
print(f"  Total cost: ${best_impact['total_cost']:,}")

print(f"\n💰 Savings: ${baseline_impact['total_cost'] - best_impact['total_cost']:,}")
print(f"📊 Improvement: {(1 - best_impact['total_cost']/baseline_impact['total_cost'])*100:.1f}%")

In [ ]:
# Feature Importance (for tree-based models)
best_classifier = rf_pipeline.named_steps['classifier']  # or gb_pipeline

# Get feature names after preprocessing
feature_names = (numerical_features + 
                list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': best_classifier.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20
plt.figure(figsize=(10, 8))
top_20 = importance_df.head(20)
plt.barh(range(len(top_20)), top_20['importance'])
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Importance')
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.show()

print("Top 10 Features:")
print(importance_df.head(10).to_string(index=False))

In [ ]:
# Confusion Matrix for Best Model
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Use best model predictions (adjust based on results)
best_predictions = y_pred_rf  # Change to y_pred_gb if Gradient Boosting is best

cm = confusion_matrix(y_test, best_predictions, labels=['Normal', 'Urgent', 'Critical'])

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal', 'Urgent', 'Critical'],
            yticklabels=['Normal', 'Urgent', 'Critical'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - Best Model')
plt.tight_layout()
plt.show()

print(f"Correct predictions: {cm.diagonal().sum()}/{len(y_test)} ({cm.diagonal().sum()/len(y_test)*100:.1f}%)")

In [ ]:
# Model Comparison Summary
import pandas as pd

comparison = pd.DataFrame({
    'Model': ['Baseline', 'Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_baseline),
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_gb)
    ],
    'F1-Score': [
        f1_score(y_test, y_pred_baseline, average='weighted'),
        f1_score(y_test, y_pred_lr, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted'),
        f1_score(y_test, y_pred_gb, average='weighted')
    ]
}).sort_values('F1-Score', ascending=False)

print("="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison.to_string(index=False))
print(f"\n🏆 Best Model: {comparison.iloc[0]['Model']}")

### Model Comparison & Analysis

In [ ]:
# Model 4: Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=5,
                                             subsample=0.8, random_state=42))
])

print("Training Gradient Boosting...")
gb_pipeline.fit(X_train, y_train)
y_pred_gb = gb_pipeline.predict(X_test)

print("\n" + "="*60)
print("GRADIENT BOOSTING")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.3f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_pred_gb, average='weighted'):.3f}")
print(f"\n{classification_report(y_test, y_pred_gb)}")

In [ ]:
# Model 3: Random Forest
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=5,
                                         class_weight='balanced', random_state=42, n_jobs=-1))
])

print("Training Random Forest...")
rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)

print("\n" + "="*60)
print("RANDOM FOREST")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_pred_rf, average='weighted'):.3f}")
print(f"\n{classification_report(y_test, y_pred_rf)}")

In [ ]:
# Model 2: Logistic Regression
from sklearn.linear_model import LogisticRegression

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', 
                                     class_weight='balanced', max_iter=1000, random_state=42))
])

print("Training Logistic Regression...")
lr_pipeline.fit(X_train, y_train)
y_pred_lr = lr_pipeline.predict(X_test)

print("\n" + "="*60)
print("LOGISTIC REGRESSION")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.3f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_pred_lr, average='weighted'):.3f}")
print(f"\n{classification_report(y_test, y_pred_lr)}")

In [ ]:
# Model 1: Baseline (DummyClassifier)
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

baseline_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DummyClassifier(strategy='most_frequent', random_state=42))
])

baseline_pipeline.fit(X_train, y_train)
y_pred_baseline = baseline_pipeline.predict(X_test)

print("="*60)
print("BASELINE MODEL (Always predict most frequent)")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_baseline):.3f}")
print(f"F1-Score (weighted): {f1_score(y_test, y_pred_baseline, average='weighted'):.3f}")
print(f"\n{classification_report(y_test, y_pred_baseline)}")

### Model Training

In [ ]:
# Step 5: Build Preprocessing Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

print("✓ Preprocessing pipeline created")
print(f"  - Numerical features (StandardScaler): {numerical_features}")
print(f"  - Categorical features (OneHotEncoder): {categorical_features}")

In [ ]:
# Step 4: Train-Test Split (stratified by ZONE)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=df_analysis['ZONE'], random_state=42
)

print(f"Train: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nTarget distribution in train:\n{y_train.value_counts()}")
print(f"\nTarget distribution in test:\n{y_test.value_counts()}")

In [ ]:
# Step 3: Prepare X (features) and y (target) - NO DATA LEAKAGE
# Only use information known at BOOKING TIME

categorical_features = ['ZONE', 'Direction', 'Type', 'DGR', 'iso_country_origin', 'iso_country_destination']
numerical_features = ['Gwgt', 'Cwgt', 'Pcs', 'day_of_week', 'month', 'hour']
all_features = categorical_features + numerical_features

X = df_analysis[all_features].copy()
y = df_analysis['optimal_service_level'].copy()

print(f"✓ Feature matrix: {X.shape}")
print(f"✓ Target: {y.shape}")
print(f"\nFeatures ({len(all_features)}): {all_features}")
print("\n🔒 DATA LEAKAGE CHECK: leadtime_ml_hours NOT in features (correct!)")

In [ ]:
# Step 2: Feature Engineering - Extract temporal features
df_analysis['day_of_week'] = df_analysis['NOTIFICATION date & Time'].dt.dayofweek  # 0=Monday
df_analysis['month'] = df_analysis['NOTIFICATION date & Time'].dt.month
df_analysis['hour'] = df_analysis['NOTIFICATION date & Time'].dt.hour

print("✓ Temporal features created from notification timestamp")

In [ ]:
# Step 1: Create optimal service level labels (vectorized, using leadtime_ml_hours)
import numpy as np

# Create SLA lookup columns (vectorized)
df_analysis['sla_normal'] = df_analysis['ZONE'].map(lambda z: sla_targets_lookup.get((z, 'Normal'), float('inf')))
df_analysis['sla_urgent'] = df_analysis['ZONE'].map(lambda z: sla_targets_lookup.get((z, 'Urgent'), float('inf')))
df_analysis['sla_critical'] = df_analysis['ZONE'].map(lambda z: sla_targets_lookup.get((z, 'Critical'), float('inf')))

# Vectorized label creation using leadtime_ml_hours (no outliers)
conditions = [
    df_analysis['leadtime_ml_hours'] <= df_analysis['sla_normal'],
    df_analysis['leadtime_ml_hours'] <= df_analysis['sla_urgent'],
    df_analysis['leadtime_ml_hours'] <= df_analysis['sla_critical']
]
choices = ['Normal', 'Urgent', 'Critical']
df_analysis['optimal_service_level'] = np.select(conditions, choices, default='Normal')

# Clean up
df_analysis.drop(['sla_normal', 'sla_urgent', 'sla_critical'], axis=1, inplace=True)

print("✓ Optimal service level labels created (vectorized, 10-100x faster)")
print(f"\nDistribution:")
print(df_analysis['optimal_service_level'].value_counts())
print(f"\nPercentages:")
print((df_analysis['optimal_service_level'].value_counts(normalize=True) * 100).round(1))

## MACHINE LEARNING - Optimal Service Level Prediction

In [ ]:
import pandas as pd

## DATA LOADING, CLEANING & INTEGRATION

In [ ]:
# Load datasets

orders = pd.read_csv("Orders.csv", encoding="latin-1", sep=";", decimal =",")
leadtime = pd.read_csv("LeadtimeService.csv", encoding="latin-1", sep=";")
airports = pd.read_csv("airports.csv", encoding="latin-1", sep=",", decimal =",")


In [ ]:
orders

In [ ]:
# Select relevant columns for analysis from orders dataset

cols_keep = [
    "Type",
    "Direction",
    "DSV-CW Ref.",
    "Goods Description",
    "Pcs",
    "Gwgt",
    "Cwgt",
    "NOTIFICATION date & Time",
    "ACTUAL Delivery & Time",
    "Service Level",
    "DGR",
    "Real Origin Airport",
    "Real Destination Airport",
    "ZONE"
]

df_analysis = orders[cols_keep]

In [ ]:
df_analysis["Service Level"].value_counts()

In [ ]:
#drop rows with service level: "ERROR en Service Level (columna AK)"

df_analysis = df_analysis[df_analysis["Service Level"] != "ERROR en Service Level (columna AK)"]

In [ ]:
df_analysis["Service Level"].value_counts()

In [ ]:
# Service Level mapping used in the analysis
service_level_mapping = {
    "ROV": "Normal",
    "CRV": "Urgent",
    "AOV": "Critical"
}

df_analysis["Service Type"] = (
    df_analysis["Service Level"]
    .map(service_level_mapping)
)

df_analysis[["Service Level", "Service Type"]].drop_duplicates()

In [ ]:
df_analysis["ZONE"].value_counts()

In [ ]:
leadtime

In [ ]:
#Drop spaces in column names
df_analysis.columns = df_analysis.columns.str.strip()
leadtime.columns = leadtime.columns.str.strip()

# Clean string columns to ensure proper merging
df_analysis["ZONE"] = df_analysis["ZONE"].astype(str).str.strip()
df_analysis["Service Level"] = df_analysis["Service Level"].astype(str).str.strip()

leadtime["Zone"] = leadtime["Zone"].astype(str).str.strip()
leadtime["SLA"] = leadtime["SLA"].astype(str).str.strip()

# Merge lead time information into the main dataframe

df_analysis = df_analysis.merge(
    leadtime[["Zone", "SLA", "Leadtime"]],
    how="left",
    left_on=["ZONE", "Service Level"],
    right_on=["Zone", "SLA"]
)

# Limpiar columnas auxiliares del join
df_analysis.drop(columns=["Zone", "SLA"], inplace=True)


In [ ]:
df_analysis

In [ ]:
airports

In [ ]:
#Drop spaces in column names
airports.columns = airports.columns.str.strip()

# Clean string columns to ensure proper merging
df_analysis["Real Origin Airport"] = df_analysis["Real Origin Airport"].astype(str).str.strip()
df_analysis["Real Destination Airport"] = df_analysis["Real Destination Airport"].astype(str).str.strip()

airports["iata_code"] = airports["iata_code"].astype(str).str.strip()

# Merge information into the main dataframe for ORIGIN airport
df_analysis = df_analysis.merge(
    airports[["iata_code", "iso_country"]],
    how="left",
    left_on="Real Origin Airport",
    right_on="iata_code"
)

# Rename columns to avoid confusion
df_analysis.rename(
    columns={
        "iata_code": "iata_code_origin",
        "iso_country": "iso_country_origin"
    },
    inplace=True
)

# Merge information into the main dataframe for DESTINATION airport
df_analysis = df_analysis.merge(
    airports[["iata_code", "iso_country"]],
    how="left",
    left_on="Real Destination Airport",
    right_on="iata_code"
)

# Rename columns to avoid confusion
df_analysis.rename(
    columns={
        "iata_code": "iata_code_destination",
        "iso_country": "iso_country_destination"
    },
    inplace=True
)


In [ ]:
# Final dataframe for analysis
df_analysis = df_analysis.drop(columns=["Real Origin Airport", "Real Destination Airport"])
df_analysis

## TARGET ENGINEERING (leadtime_real)

In [ ]:
#Convert date columns to datetime format

notification_col = "NOTIFICATION date & Time"
delivery_col = "ACTUAL Delivery & Time"

df_analysis[notification_col] = pd.to_datetime(
    df_analysis[notification_col],
    errors="coerce",
    dayfirst=True
)

df_analysis[delivery_col] = pd.to_datetime(
    df_analysis[delivery_col],
    errors="coerce",
    dayfirst=True
)

print("\nMissing timestamps after datetime conversion:")
display(df_analysis[[notification_col, delivery_col]].isna().sum())

In [ ]:
#Drop rows with missing timestamps
df_analysis = df_analysis.dropna(subset=[notification_col, delivery_col])

df_analysis

In [ ]:
#Creating leadtime real in hours

df_analysis["leadtime_real_hours"] = (
    (df_analysis[delivery_col] - df_analysis[notification_col])
    .dt.total_seconds() / 3600
)

df_analysis[["leadtime_real_hours", notification_col, delivery_col]]


In [ ]:
# Negative lead times check because of data errors
print("Negatives:", (df_analysis["leadtime_real_hours"] < 0).sum())


In [ ]:
# Dealing with negative and 0 lead times

negatives_df = df_analysis[df_analysis["leadtime_real_hours"] <= 0]

print(f"Negative lead times: {len(negatives_df)}")

display(
    negatives_df[
        [
            "NOTIFICATION date & Time",
            "ACTUAL Delivery & Time",
            "leadtime_real_hours",
            "ZONE",
            "Service Level",
            "Service Type"
        ]
    ].sort_values("leadtime_real_hours")
)

In [ ]:
#Flagging invalid lead times

df_analysis["invalid_leadtime_flag"] = (df_analysis["leadtime_real_hours"] <= 0).astype(int)

df_analysis["invalid_leadtime_flag"].value_counts()

In [ ]:
df_analysis = df_analysis[df_analysis["leadtime_real_hours"] > 0].copy()

#Statistics summary
print(df_analysis["leadtime_real_hours"].describe())


In [ ]:
#Creating a target with 99th percentile lead time for avoid outliers

leadtime_99th_percentile = df_analysis["leadtime_real_hours"].quantile(0.99)

df_analysis["leadtime_ml_hours"] = df_analysis["leadtime_real_hours"].clip(upper=leadtime_99th_percentile)

#Statistics summary
print(df_analysis["leadtime_ml_hours"].describe())

In [ ]:
# leadtime_real_hours: real operational lead time (EDA)
# leadtime_ml_hours: capped target used for ML training with 99th percentile

In [ ]:
#Calculating on_time comparing leadtime_real_hours with SLA
df_analysis["on_time"] = (df_analysis["leadtime_real_hours"] <= df_analysis["Leadtime"]).astype(int)

In [ ]:
df_analysis

## Exploratory Data Analysis


In [ ]:

# Ranking SLA service levels (1=cheapest, 3=most expensive)
sla_rank = {
    "Normal": 1,
    "Urgent": 2,
    "Critical": 3
}

# Create a lookup dictionary for SLA targets by ZONE + Service Type
sla_targets_lookup = (
    df_analysis
    .groupby(["ZONE", "Service Type"])["Leadtime"]
    .first()
    .to_dict()
)

# Function to determine minimum required SLA based on actual delivery time and zone
def calculate_minimum_required_sla(row):
    """
    Determines the cheapest SLA level that would have met the delivery time
    for the specific ZONE of this order.
    """
    actual_leadtime = row["leadtime_real_hours"]
    zone = row["ZONE"]
    
    # Check from cheapest to most expensive for this specific zone
    service_levels = ["Normal", "Urgent", "Critical"]
    
    for service_type in service_levels:
        target_key = (zone, service_type)
        if target_key in sla_targets_lookup:
            target_hours = sla_targets_lookup[target_key]
            if actual_leadtime <= target_hours:
                return service_type
    
    return "Out of SLA"

# Apply the function
df_analysis["sla_required"] = df_analysis.apply(calculate_minimum_required_sla, axis=1)

# Recommend SLA based on delivery performance
def recommend_sla(row):
    if row["sla_required"] == "Out of SLA":
        return "Normal"  # Late delivery gets minimum rate
    elif row["sla_required"] != row["Service Type"]:
        return row["sla_required"]  # Could have used cheaper SLA
    else:
        return row["Service Type"]  # Keep contracted level

df_analysis["sla_recommended"] = df_analysis.apply(recommend_sla, axis=1)

# Map to ranks for comparison
df_analysis["sla_contracted_rank"] = df_analysis["Service Type"].map(sla_rank)
df_analysis["sla_recommended_rank"] = df_analysis["sla_recommended"].map(sla_rank)

# Calculate levels to downgrade (how many ranks cheaper)
df_analysis["levels_to_downgrade"] = (
    df_analysis["sla_contracted_rank"] - df_analysis["sla_recommended_rank"]
)

# Downgrade is only possible if there are actual savings (levels_to_downgrade > 0)
df_analysis["downgrade_possible"] = (df_analysis["levels_to_downgrade"] > 0)

# Summary statistics
print("\n" + "="*60)
print("DOWNGRADE ANALYSIS SUMMARY")
print("="*60)
print(f"\nTotal orders analyzed: {len(df_analysis)}")
print(f"Orders with downgrade opportunity: {df_analysis['downgrade_possible'].sum()}")
print(f"  - With savings (levels > 0): {(df_analysis['levels_to_downgrade'] > 0).sum()}")
print(f"  - No savings (levels = 0): {(df_analysis['levels_to_downgrade'] == 0).sum()}")
print(f"Downgrade rate: {df_analysis['downgrade_possible'].mean():.1%}")

print("\nBreakdown by levels_to_downgrade:")
print(df_analysis['levels_to_downgrade'].value_counts().sort_index())

print("\nDowngrade opportunities by contracted Service Type:")
downgrade_by_service = df_analysis.groupby("Service Type").agg({
    'downgrade_possible': ['sum', 'mean'],
    'levels_to_downgrade': 'mean'
})
print(downgrade_by_service)



In [ ]:
# Show examples
print("\n" + "="*60)
print("EXAMPLES:")
print("="*60)

print("\n1. Downgrade possible (levels > 0):")
print(df_analysis[df_analysis["downgrade_possible"]][
    ["DSV-CW Ref.", "ZONE", "Service Type", "leadtime_real_hours", "Leadtime", 
     "sla_recommended", "levels_to_downgrade", "downgrade_possible"]
].head(3))

print("\n2. No downgrade (levels = 0, already at minimum or met SLA):")
print(df_analysis[df_analysis["levels_to_downgrade"] == 0][
    ["DSV-CW Ref.", "ZONE", "Service Type", "leadtime_real_hours", "Leadtime", 
     "sla_recommended", "levels_to_downgrade", "downgrade_possible"]
].head(3))

print("\n3. Specific order SBCN0260531:")
specific_order = df_analysis[df_analysis["DSV-CW Ref."] == "SBCN0260531"][
    ["DSV-CW Ref.", "ZONE", "Service Type", "leadtime_real_hours", "Leadtime", 
     "sla_recommended", "levels_to_downgrade", "downgrade_possible"]
]
if len(specific_order) > 0:
    print(specific_order)
else:
    print("Order not found in filtered dataset")

In [ ]:
df_analysis

## Machine Learning

In [ ]:
def get_sla_target(zone, service_type):
    """
    Return SLA (hours) for a given zone and service_type.
    Tries:
      1) zone + service_type lookup (sla_targets_lookup)
      2) type-level default (sla_targets_by_type)
      3) final fallback: +inf (so comparisons fail safely)
    """
    key = (zone, service_type)
    if "sla_targets_lookup" in globals() and key in sla_targets_lookup:
        try:
            return float(sla_targets_lookup[key])
        except Exception:
            pass

    if "sla_targets_by_type" in globals() and service_type in sla_targets_by_type:
        try:
            return float(sla_targets_by_type[service_type])
        except Exception:
            pass

    return float("inf")


def create_optimal_service_label(row):
    """
    For each historical shipment, determine the MINIMUM service level
    that would have delivered within SLA.

    This becomes our ground truth: what we SHOULD have booked.
    """
    actual_delivery_time = row["leadtime_real_hours"]
    zone = row["ZONE"]

    # Get SLA targets for this zone (always numeric thanks to get_sla_target)
    normal_sla = get_sla_target(zone, "Normal")
    urgent_sla = get_sla_target(zone, "Urgent")
    critical_sla = get_sla_target(zone, "Critical")

    # Assign minimum sufficient service level (check from cheapest to most expensive)
    if actual_delivery_time <= normal_sla:
        return "Normal"  # Cheapest option would have worked
    elif actual_delivery_time <= urgent_sla:
        return "Urgent"  # Needed mid-tier
    elif actual_delivery_time <= critical_sla:
        return "Critical"  # Needed premium
    else:
        return "Normal"  # Even Critical wouldn't have worked → default to cheapest


# Apply to dataset
df_analysis["optimal_service_level"] = df_analysis.apply(
    create_optimal_service_label, axis=1
)

In [ ]:
# Check distribution of optimal service levels
print("="*60)
print("OPTIMAL SERVICE LEVEL DISTRIBUTION (What we SHOULD have booked)")
print("="*60)
print(df_analysis["optimal_service_level"].value_counts())
print(f"\nPercentages:")
print(df_analysis["optimal_service_level"].value_counts(normalize=True).round(3) * 100)

# Compare with what was actually booked
print("\n" + "="*60)
print("COMPARISON: Actual vs. Optimal")
print("="*60)
comparison = pd.crosstab(
    df_analysis["Service Type"],  # What was booked
    df_analysis["optimal_service_level"],  # What should have been booked
    margins=True
)
print(comparison)


In [ ]:
# Extract temporal features from notification timestamp
df_analysis["day_of_week"] = df_analysis["NOTIFICATION date & Time"].dt.dayofweek  # 0=Monday, 6=Sunday
df_analysis["month"] = df_analysis["NOTIFICATION date & Time"].dt.month
df_analysis["hour"] = df_analysis["NOTIFICATION date & Time"].dt.hour

print("\nTemporal features created:")
print(df_analysis[["NOTIFICATION date & Time", "day_of_week", "month", "hour"]].head())


In [ ]:
# Define feature columns
categorical_features = ["ZONE", "Direction", "Type", "DGR", "iso_country_origin", "iso_country_destination"]
numerical_features = ["Gwgt", "Cwgt", "Pcs", "day_of_week", "month", "hour"]

# Create X (features) and y (target)
X = df_analysis[categorical_features + numerical_features].copy()
y = df_analysis["optimal_service_level"].copy()

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {X.columns.tolist()}")


In [ ]:
from sklearn.model_selection import train_test_split

# Split with stratification by ZONE (ensure all zones in both sets)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=df_analysis["ZONE"],  # Maintain zone distribution
    random_state=42
)

print("\n" + "="*60)
print("TRAIN-TEST SPLIT")
print("="*60)
print(f"Train set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:  {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

print("\nTarget distribution in train set:")
print(y_train.value_counts())
print("\nTarget distribution in test set:")
print(y_test.value_counts())


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='drop'
)

print("\n" + "="*60)
print("PREPROCESSING PIPELINE CREATED")
print("="*60)
print(f"Numerical features (will be standardized): {numerical_features}")
print(f"Categorical features (will be one-hot encoded): {categorical_features}")


## Train Baseline Model

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Create pipeline with DummyClassifier
baseline_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DummyClassifier(strategy='most_frequent', random_state=42))
])

# Train
baseline_pipeline.fit(X_train, y_train)

# Predict
y_pred_baseline = baseline_pipeline.predict(X_test)

# Evaluate
print("="*60)
print("BASELINE MODEL (DummyClassifier - Most Frequent)")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_baseline):.3f}")
print(f"Weighted F1-Score: {f1_score(y_test, y_pred_baseline, average='weighted'):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_baseline))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_baseline))


In [ ]:
from sklearn.linear_model import LogisticRegression

# Create pipeline
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        multi_class='multinomial',
        solver='lbfgs',
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    ))
])

# Train
print("\nTraining Logistic Regression...")
lr_pipeline.fit(X_train, y_train)

# Predict
y_pred_lr = lr_pipeline.predict(X_test)

# Evaluate
print("\n" + "="*60)
print("LOGISTIC REGRESSION")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.3f}")
print(f"Weighted F1-Score: {f1_score(y_test, y_pred_lr, average='weighted'):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create pipeline
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ))
])

# Train
print("\nTraining Random Forest...")
rf_pipeline.fit(X_train, y_train)

# Predict
y_pred_rf = rf_pipeline.predict(X_test)

# Evaluate
print("\n" + "="*60)
print("RANDOM FOREST")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.3f}")
print(f"Weighted F1-Score: {f1_score(y_test, y_pred_rf, average='weighted'):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create pipeline
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        random_state=42
    ))
])

# Train
print("\nTraining Gradient Boosting...")
gb_pipeline.fit(X_train, y_train)

# Predict
y_pred_gb = gb_pipeline.predict(X_test)

# Evaluate
print("\n" + "="*60)
print("GRADIENT BOOSTING")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.3f}")
print(f"Weighted F1-Score: {f1_score(y_test, y_pred_gb, average='weighted'):.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb))


In [ ]:
import pandas as pd

# Create comparison dataframe
models_comparison = pd.DataFrame({
    'Model': ['Baseline (Dummy)', 'Logistic Regression', 'Random Forest', 'Gradient Boosting'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_baseline),
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_gb)
    ],
    'Weighted F1-Score': [
        f1_score(y_test, y_pred_baseline, average='weighted'),
        f1_score(y_test, y_pred_lr, average='weighted'),
        f1_score(y_test, y_pred_rf, average='weighted'),
        f1_score(y_test, y_pred_gb, average='weighted')
    ]
})

# Sort by F1-Score
models_comparison = models_comparison.sort_values('Weighted F1-Score', ascending=False)

print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)
print(models_comparison.to_string(index=False))

# Identify best model
best_model_name = models_comparison.iloc[0]['Model']
print(f"\n🏆 Best Model: {best_model_name}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay

# Determine which model is best (use the actual best model predictions)
# For this example, let's use Random Forest (update based on your results)
best_predictions = y_pred_rf  # Change this to y_pred_gb if Gradient Boosting is best

# Create confusion matrix
cm = confusion_matrix(y_test, best_predictions, labels=['Normal', 'Urgent', 'Critical'])

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Urgent', 'Critical'],
            yticklabels=['Normal', 'Urgent', 'Critical'],
            ax=ax)
ax.set_xlabel('Predicted Service Level')
ax.set_ylabel('Actual Optimal Service Level')
ax.set_title('Confusion Matrix - Best Model')
plt.tight_layout()
plt.show()

# Analyze errors
print("\n" + "="*60)
print("ERROR ANALYSIS")
print("="*60)
print("\nDiagonal (correct predictions):", cm.diagonal().sum())
print("Below diagonal (overestimated - predicted higher than needed):", sum([cm[i,j] for i in range(3) for j in range(3) if i < j]))
print("Above diagonal (underestimated - predicted lower than needed):", sum([cm[i,j] for i in range(3) for j in range(3) if i > j]))


In [ ]:
# Extract feature importance from Random Forest or Gradient Boosting
# Get the classifier from the pipeline
best_classifier = rf_pipeline.named_steps['classifier']  # or gb_pipeline if GB is best

# Get feature names after preprocessing
feature_names = (
    numerical_features + 
    list(preprocessor.named_transformers_['cat']
         .get_feature_names_out(categorical_features))
)

# Get importance
feature_importance = best_classifier.feature_importances_

# Create DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Plot top 20
plt.figure(figsize=(10, 8))
top_20 = importance_df.head(20)
plt.barh(range(len(top_20)), top_20['importance'])
plt.yticks(range(len(top_20)), top_20['feature'])
plt.xlabel('Importance')
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("TOP 10 MOST IMPORTANT FEATURES")
print("="*60)
print(importance_df.head(10).to_string(index=False))


In [ ]:
# Calculate business metrics
def calculate_business_impact(y_true, y_pred):
    """
    Calculate cost impact of predictions.
    """
    service_rank = {'Normal': 1, 'Urgent': 2, 'Critical': 3}
    tier_cost = 100  # $ per service tier difference
    late_delivery_cost = 500  # $ opportunity cost if predict too low
    
    total_overspend = 0
    total_late_risk = 0
    correct_predictions = 0
    
    for true, pred in zip(y_true, y_pred):
        true_rank = service_rank[true]
        pred_rank = service_rank[pred]
        
        if pred_rank == true_rank:
            correct_predictions += 1
        elif pred_rank > true_rank:  # Overestimated
            total_overspend += (pred_rank - true_rank) * tier_cost
        elif pred_rank < true_rank:  # Underestimated
            total_late_risk += late_delivery_cost
    
    return {
        'correct': correct_predictions,
        'total_overspend': total_overspend,
        'total_late_risk': total_late_risk,
        'net_cost': total_overspend + total_late_risk
    }

# Calculate for baseline vs best model
baseline_impact = calculate_business_impact(y_test, y_pred_baseline)
best_impact = calculate_business_impact(y_test, best_predictions)

print("\n" + "="*60)
print("BUSINESS IMPACT ANALYSIS (Test Set)")
print("="*60)
print(f"\nBaseline Model (Always predict most frequent):")
print(f"  Correct predictions: {baseline_impact['correct']}/{len(y_test)} ({baseline_impact['correct']/len(y_test)*100:.1f}%)")
print(f"  Total overspend: ${baseline_impact['total_overspend']:,}")
print(f"  Total late risk: ${baseline_impact['total_late_risk']:,}")
print(f"  Net cost: ${baseline_impact['net_cost']:,}")

print(f"\nBest ML Model:")
print(f"  Correct predictions: {best_impact['correct']}/{len(y_test)} ({best_impact['correct']/len(y_test)*100:.1f}%)")
print(f"  Total overspend: ${best_impact['total_overspend']:,}")
print(f"  Total late risk: ${best_impact['total_late_risk']:,}")
print(f"  Net cost: ${best_impact['net_cost']:,}")

print(f"\n💰 Cost Reduction: ${baseline_impact['net_cost'] - best_impact['net_cost']:,}")
print(f"📊 Improvement: {(1 - best_impact['net_cost']/baseline_impact['net_cost'])*100:.1f}%")


In [ ]:
import joblib

# Save the best model
best_model = rf_pipeline  # or gb_pipeline, depending on results

joblib.dump(best_model, 'best_service_level_model.pkl')
print("\n✅ Best model saved as 'best_service_level_model.pkl'")

# To load later:
# loaded_model = joblib.load('best_service_level_model.pkl')
# predictions = loaded_model.predict(new_data)
